# NPL Generative Model in Chinese - Su Shi

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

In [1]:
import spacy
import jieba

from spacy.util import get_lang_class
nlp = get_lang_class('zh')

## Who is Su Shi?
https://en.wikipedia.org/wiki/Su_Shi

# SpaCy
https://spacy.io/usage/adding-languages

## Setup

We're going to download the collected works of Nietzsche to use as our data for this class.

In [7]:
import sys
sys.path.append('C://Users/sarad/Documents/fastai/')

In [55]:
PATH='C://Users/sarad/Documents/fastai/courses/dl1/data/SuShi/'

In [8]:
file = 'C://Users/sarad/Documents/fastai/courses/dl1/data/SuShi/SuShi.txt'

#get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')
text = open(file, encoding="UTF-8").read()
print('corpus length:', len(text))

corpus length: 237313


In [9]:
text[:400]

'\ufeff\n【辛醜十一月十九日既與子由別於鄭州西門之外馬上賦詩一篇寄之】\n不飲胡為醉兀兀，此心已逐歸鞍發。歸人猶自念庭闈，今我何以慰寂寞。登高回\n首坡壟隔，惟見烏帽出複沒。苦寒念爾衣裳薄，獨騎瘦馬踏殘月。路人行歌居人\n樂，僮僕怪我苦淒惻。亦知人生要有別，但恐歲月去飄忽。寒燈相對記疇昔，夜\n雨何時聽蕭瑟。君知此意不可忘，慎勿苦愛高官職。（嘗有夜雨對床之言，故雲\n\n\x0c爾。）\n【和子由澠池懷舊】\n人生到處知何似，應似飛鴻踏雪泥。泥上偶然留指爪，鴻飛那複計東西。老僧已\n死成新塔，壞壁無由見舊題。往日崎嶇還記否，路長人困蹇驢嘶。（往歲，馬死\n於二陵，騎驢至澠池。）\n【次韻和劉京兆石林亭之作石本唐苑中物散流民間劉購得之】\n都城日荒廢，往事不可還。惟余故苑石，漂散向人間。公來始購蓄，不憚道裏艱。\n盡従塵埃中，來對冰雪顏。瘦骨拔凜凜，蒼根漱潺潺。唐人惟奇章，好石古莫攀。\n盡令屬牛氏，刻鑿紛班班。嗟此本何常，聚散實'

In [10]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 5076


Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [11]:
chars.insert(0, "\0")

''.join(chars[1:-6])

'\n\x0c 0123456789?AS{}ω“”•│◎、。々《》【】〔〕〗一丁七丈三上下不且丕世丘丙丞並丫中丸丹主乃久么之乍乎乏乖乘乙九乞也乳乾亂了予事二于互五井些亞亡亢交亥亦亨享京亭亮亹人什仁仄仇今介仍仕他仗付仙仝仞代令以仰仲任仿企伊伋伍伎伏伐休伯伴伶伸伺似伽但佇佈位低住佐佔何佗佘余佚佛作佞佩佯佳併使侄來例侍侏侑侔侖侗供依侮侯侵侶便促俄俊俎俗俘俚俛俜保俞俟俠信修俯俱俸倅倆倉個倍倏倒倕倖倘候倚倛倜借倡倦倩倪倫倭值偃假偈偉偏偕停健側偶偷偽傅傍傑傖傘備催傭傲傳傴債傷傾僂僅像僑僕僚僛僥僦僧僬僭僮僵價僻儀儂儆儇儉儋儒儔儕儘償優儲儺儻儼兀元兄充兆先光克免兒兔兕党兜兢入內全兩八公六兮共兵其具典兼冀冉冊再冒冕冗冠冤冥冬冰冶冷冽准凋凍凜凝凡凰凱凶凸出函刀刁刃分切刈刊刑刓刖列初判別利刪刮到刲刳制刷券刹刺刻剃則削前剔剖剗剛剝剡剩剪副割創剽劃劄劇劈劉劍劑劚力功加劣助努劫劬劾勁勃勇勉勍勒動勖務勝勞勢勤勳勸勺勻勾勿包匆匊匏匕化北匙匝匠匡匣匪匯匱匹匿區十千卅升午卉半卑卒卓南博卜卞占卣卮卯印危即卵卷卻卿厄厔厖厚原厥厭厲去參又叉及友反叔取受叛叟叢口古句叨叩只叫召可台叱史右叵司吃各合吉吊同名后吏吐向吒君吞吟吠否含吭吳吷吸吹吻吼吾呀呂呈告呦周呱味呵呶呷呻呼命咀咄和咍咎咒咨咫咬咮咳咸咻咽咿哀品哂哄哇哈哉員哢哥哦哭哲哺哽唇唐唧售唯唱唳唷唾啄啅商問啖啜啞啟啻啼啾喁善喉喊喑喘喙喚喜喟喧喪喬單喻嗄嗅嗒嗔嗚嗜嗟嗣嗤嗾嘅嘆嘉嘔嘗嘩嘬嘯嘲嘵嘶嘹噀噂噍噎噓噞噤器噪噫噬噱噲噴嚅嚇嚌嚏嚬嚴嚶嚼囀囁囂囊囑囚四回因困囷固囿圃圈圉國圍園圓圖團圜土圠在圭圯地址均坊坌坎坐坑坡坤坦坯坰坱坳坷坼垂型垠垢垣埃埋城埒域埭執培基埽堂堅堆堇堙堠堤堧堪堯堰報場堵塉塊塌塑塔塗塘塚塞塢填塯塵塹塼塿墀墁境墓墜增墟墦墨墮墳墾壁壅壇壑壓壖壘壙壚壞壟壤士壬壯壺壽夏夔夕外夙多夜夢夥大天太夫夭央失夷夾奄奇奈奉奏契奔奕奚奢奧奩奪奮女奴奸好如妃妄妉妍妒妓妖妙妝妤妨妻妾姆姊始姐姑姓委姚姜姝姥姨姬姹姻姿威娉娑娘娛娜娟娥娩娶婀婁婆婉婕婚婞婢婦婭婷婺婿媒媚媧媮媸媼媾嫁嫂嫋嫌嫚嫣嫦嫩嫫嫮嫵嬈嬉嬋嬌嬛嬰孌子孔孕字存孚孜孝孟季孤孥孩孫孰孱學孺孽它宅宇守安宋完宏宕宗官宙定宛宜客宣室宥宦宮宰害宴宵家宸容宿寂寄寅密寇富寐寒寓寘寞察寡寢寤寥實寧寨審寫寬寮寰寵寶寸寺封射將專尉尊尋對導小少尖尚尤尫就尹尺尻尼尾局居屈屋屍屏屐屑展屟屠屢屣層履屨屬屭屯山屹岌岏岐岑岡岢岣岩岫岱岳岷岸峒峙峨峭峰峴

Map from chars to indices and back again

In [12]:
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

*idx* will be the data we use from now own - it simply converts all the characters to their index (based on the mapping above)

In [13]:
idx = [char_indices[c] for c in text]

idx[:10]

[5066, 1, 30, 4196, 4369, 416, 35, 1723, 416, 65]

In [14]:
''.join(indices_char[i] for i in idx[:70])

'\ufeff\n【辛醜十一月十九日既與子由別於鄭州西門之外馬上賦詩一篇寄之】\n不飲胡為醉兀兀，此心已逐歸鞍發。歸人猶自念庭闈，今我何以慰寂寞。登高回\n首'

## Three char model

### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [15]:
cs=3
c1_dat = [idx[i]   for i in range(0, len(idx)-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-cs, cs)]

Our inputs

In [16]:
x1 = np.stack(c1_dat)
x2 = np.stack(c2_dat)
x3 = np.stack(c3_dat)

Our output

In [17]:
y = np.stack(c4_dat)

The first 4 inputs and outputs

In [18]:
x1[:4], x2[:4], x3[:4]

(array([5066, 4196,   35,   65]),
 array([   1, 4369, 1723, 1632]),
 array([  30,  416,  416, 1631]))

In [19]:
y[:4]

array([4196,   35,   65, 3395])

In [20]:
x1.shape, y.shape

((79104,), (79104,))

### Create and train model

Pick a size for our hidden state

In [21]:
n_hidden = 256

The number of latent factors to create (i.e. the size of the embedding matrix)

In [22]:
n_fac = 42

In [23]:
class Char3Model(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)

        # The 'green arrow' from our diagram - the layer operation from input to hidden
        self.l_in = nn.Linear(n_fac, n_hidden)

        # The 'orange arrow' from our diagram - the layer operation from hidden to hidden
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        
        # The 'blue arrow' from our diagram - the layer operation from hidden to output
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, c1, c2, c3):
        in1 = F.relu(self.l_in(self.e(c1)))
        in2 = F.relu(self.l_in(self.e(c2)))
        in3 = F.relu(self.l_in(self.e(c3)))
        
        h = V(torch.zeros(in1.size()).cuda())
        h = F.tanh(self.l_hidden(h+in1))
        h = F.tanh(self.l_hidden(h+in2))
        h = F.tanh(self.l_hidden(h+in3))
        
        return F.log_softmax(self.l_out(h))

In [24]:
md = ColumnarModelData.from_arrays('.', [-1], np.stack([x1,x2,x3], axis=1), y, bs=512)

In [25]:
m = Char3Model(vocab_size, n_fac).cuda()

In [26]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [27]:
opt = optim.Adam(m.parameters(), 1e-2)

In [28]:
fit(m, md, 5, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=5), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      7.27401    1.014222  
    1      7.061869   2.067131                                                                                         
    2      6.817791   1.432975                                                                                         
    3      6.619451   1.327675                                                                                         
    4      6.398521   1.210383                                                                                         



[1.2103834]

In [29]:
set_lrs(opt, 0.001)

In [30]:
fit(m, md, 5, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=5), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      5.134042   1.306654  
    1      4.795378   1.256146                                                                                         
    2      4.572484   1.190495                                                                                         
    3      4.398297   1.221111                                                                                         
    4      4.254784   1.288732                                                                                         



[1.2887316]

### Test model

In [31]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [34]:
get_next('我一大')

'常'

In [35]:
get_next('等一等')

'。'

In [37]:
get_next('此一凡')

'，'

In [40]:
get_next('予事二')

'首'

## Our first RNN!

### Create inputs

This is the size of our unrolled RNN.

In [41]:
cs=8

For each of 0 through 7, create a list of every 8th character with that starting point. These will be the 8 inputs to out model.

In [42]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs)]

Then create a list of the next character in each of these series. This will be the labels for our model.

In [43]:
c_out_dat = [idx[j+cs] for j in range(len(idx)-cs)]

In [44]:
xs = np.stack(c_in_dat, axis=0)

In [45]:
xs.shape

(237305, 8)

In [46]:
y = np.stack(c_out_dat)

So each column below is one series of 8 characters from the text.

In [47]:
xs[:cs,:cs]

array([[5066,    1,   30, 4196, 4369,  416,   35, 1723],
       [   1,   30, 4196, 4369,  416,   35, 1723,  416],
       [  30, 4196, 4369,  416,   35, 1723,  416,   65],
       [4196, 4369,  416,   35, 1723,  416,   65, 1632],
       [4369,  416,   35, 1723,  416,   65, 1632, 1631],
       [ 416,   35, 1723,  416,   65, 1632, 1631, 3395],
       [  35, 1723,  416,   65, 1632, 1631, 3395,  859],
       [1723,  416,   65, 1632, 1631, 3395,  859, 2613]])

...and this is the next character after each sequence.

In [48]:
y[:cs]

array([ 416,   65, 1632, 1631, 3395,  859, 2613,  333])

### Create and train model

In [49]:
val_idx = get_cv_idxs(len(idx)-cs-1)

In [50]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [51]:
class CharLoopModel(nn.Module):
    # This is an RNN!
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = F.relu(self.l_in(self.e(c)))
            h = F.tanh(self.l_hidden(h+inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [52]:
m = CharLoopModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [53]:
fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      6.984252   6.948141  



[6.948141]

In [54]:
set_lrs(opt, 0.001)

In [55]:
fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      6.030232   6.178338  



[6.178338]

In [56]:
class CharLoopConcatModel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = torch.cat((h, self.e(c)), 1)
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [57]:
m = CharLoopConcatModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [58]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [59]:
fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      6.629756   6.565658  



[6.565658]

In [60]:
set_lrs(opt, 1e-4)

In [61]:
fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      6.406258   6.425553  



[6.4255533]

### Test model

In [62]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [64]:
get_next('一直到天色灰沉')

'。'

In [67]:
get_next('恭喜發財身體健')

'。'

## RNN with pytorch

In [68]:
class CharRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        
        return F.log_softmax(self.l_out(outp[-1]), dim=-1)

In [69]:
m = CharRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [70]:
it = iter(md.trn_dl)
*xs,yt = next(it)

In [71]:
t = m.e(V(torch.stack(xs)))
t.size()

torch.Size([8, 512, 42])

In [72]:
ht = V(torch.zeros(1, 512,n_hidden))
outp, hn = m.rnn(t, ht)
outp.size(), hn.size()

(torch.Size([8, 512, 256]), torch.Size([1, 512, 256]))

In [73]:
t = m(*V(xs)); t.size()

torch.Size([512, 5076])

In [74]:
fit(m, md, 4, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      6.197723   6.148125  
    1      5.918771   5.95801                                                                                          
    2      5.706629   5.860617                                                                                         
    3      5.523365   5.798117                                                                                         



[5.798117]

In [75]:
set_lrs(opt, 1e-4)

In [76]:
fit(m, md, 2, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      5.219604   5.771392  
    1      5.193476   5.764883                                                                                         



[5.764883]

### Test model

In [77]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [78]:
get_next('急性')

'山'

In [79]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [82]:
get_next_n('予事', 10)

'予事，不復識詩全集卷二十'

In [83]:
get_next_n('急性', 30)

'急性山川，未暇年來此詩，此身，但恐我亦何妨，但恐我亦何妨，但恐我'

## Multi-output model

### Setup

Let's take non-overlapping sets of characters this time

In [84]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(idx)-cs-1, cs)]

Then create the exact same thing, offset by 1, as our labels

In [85]:
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs, cs)]

In [86]:
xs = np.stack(c_in_dat)
xs.shape

(29663, 8)

In [87]:
ys = np.stack(c_out_dat)
ys.shape

(29663, 8)

In [88]:
xs[:cs,:cs]

array([[5066,    1,   30, 4196, 4369,  416,   35, 1723],
       [ 416,   65, 1632, 1631, 3395,  859, 2613,  333],
       [1616, 4337, 1039, 3842, 4485,   58,  748, 4751],
       [  40, 4049, 3897,   35, 2997,  908,   58,   31],
       [   1,   42, 4715, 3325, 2373, 4363,  264,  264],
       [5070, 1993, 1208, 1049, 4236, 1999, 4641, 2680],
       [  26, 1999,   92, 2496, 3384, 1220, 1112, 4511],
       [5070,   97, 1361,  137,  109, 1323,  907,  917]])

In [89]:
ys[:cs,:cs]

array([[   1,   30, 4196, 4369,  416,   35, 1723,  416],
       [  65, 1632, 1631, 3395,  859, 2613,  333, 1616],
       [4337, 1039, 3842, 4485,   58,  748, 4751,   40],
       [4049, 3897,   35, 2997,  908,   58,   31,    1],
       [  42, 4715, 3325, 2373, 4363,  264,  264, 5070],
       [1993, 1208, 1049, 4236, 1999, 4641, 2680,   26],
       [1999,   92, 2496, 3384, 1220, 1112, 4511, 5070],
       [  97, 1361,  137,  109, 1323,  907,  917,   26]])

### Create and train model

In [90]:
val_idx = get_cv_idxs(len(xs)-cs-1)

In [91]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, ys, bs=512)

In [92]:
class CharSeqRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        return F.log_softmax(self.l_out(outp), dim=-1)

In [93]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [94]:
it = iter(md.trn_dl)
*xst,yt = next(it)

In [95]:
def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size()
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1,nh), targ)

In [96]:
fit(m, md, 4, opt, nll_loss_seq)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      7.135257   6.813327  
    1      6.836601   6.67459                                                                                          
    2      6.682877   6.587859                                                                                         
    3      6.571463   6.520045                                                                                         



[6.520045]

In [97]:
set_lrs(opt, 1e-4)

In [98]:
fit(m, md, 1, opt, nll_loss_seq)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      6.430997   6.495611  



[6.495611]

### Identity init!

In [99]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [100]:
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))


    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.cuda.FloatTensor of size 256x256 (GPU 0)]

In [101]:
fit(m, md, 4, opt, nll_loss_seq)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      6.998294   6.873904  
    1      6.799324   6.734665                                                                                         
    2      6.615573   6.573322                                                                                         
    3      6.412263   6.400746                                                                                         



[6.400746]

In [102]:
set_lrs(opt, 1e-3)

In [103]:
fit(m, md, 4, opt, nll_loss_seq)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      5.932524   6.213053  
    1      5.868279   6.187963                                                                                         
    2      5.818639   6.169723                                                                                         
    3      5.770109   6.150766                                                                                         



[6.150766]

## Stateful model

### Setup

In [124]:
import spacy
nlp = spacy.load('en') 
#nlp = spacy.load('en_core_web_sm')
#nlp = spacy.load('en_core_web_sm-1.2.0')

#import en_core_web_md



    Only loading the 'en' tokenizer.



In [125]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

In [127]:


#from spacy.lang.zh import Chinese
#nlp = Chinese()

In [128]:
PATH = 'C://Users/sarad/Documents/fastai/courses/dl1/data/SuShi/SuShi.txt'

TRN_PATH = file
VAL_PATH = file
TRN = file
VAL = file

#%ls {PATH}

In [129]:
%ls {PATH}

Invalid switch - "/".


In [130]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=8; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(445, 3698, 1, 228479)

In [131]:
??LanguageModelData

### RNN

In [132]:
class CharSeqStatefulRnn(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        self.vocab_size = vocab_size
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [133]:
m = CharSeqStatefulRnn(md.nt, n_fac, 512).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [134]:
fit(m, md, 4, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      6.098327   6.006408  
    1      5.766559   5.658784                                                                                         
    2      5.545071   5.419169                                                                                         
    3      5.324456   5.232852                                                                                         



[5.2328515]

In [135]:
set_lrs(opt, 1e-4)

%timeit fit(m, md, 4, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      5.06208    5.153009  
    1      5.062025   5.127876                                                                                         
    2      5.024437   5.106846                                                                                         
    3      5.040597   5.088375                                                                                         



HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      5.028587   5.069773  
    1      5.008036   5.050532                                                                                         
    2      4.987772   5.036982                                                                                         
    3      4.977088   5.016352                                                                                         



HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

...

epoch      trn_loss   val_loss                                                                                         
    0      4.79973    4.813143  
    1      4.778301   4.803526                                                                                         
    2      4.773802   4.783237                                                                                         
    3      4.725565   4.772972                                                                                         



HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      4.748817   4.758426  
    1      4.733264   4.74334                                                                                          
    2      4.708388   4.732416                                                                                         
    3      4.702018   4.721746                                                                                         



HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      4.684811   4.711418  
    1      4.688835   4.693983                                                                                         
    2      4.672451   4.683557                                                                                         
    3      4.653609   4.675121                                                                                         

19.4 s ± 250 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### RNN loop

In [136]:
# From the pytorch source

def RNNCell(input, hidden, w_ih, w_hh, b_ih, b_hh):
    return F.tanh(F.linear(input, w_ih, b_ih) + F.linear(hidden, w_hh, b_hh))

In [137]:
class CharSeqStatefulRnn2(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNNCell(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp = []
        o = self.h
        for c in cs: 
            o = self.rnn(self.e(c), o)
            outp.append(o)
        outp = self.l_out(torch.stack(outp))
        self.h = repackage_var(o)
        return F.log_softmax(outp, dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [138]:
m = CharSeqStatefulRnn2(md.nt, n_fac, 512).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [139]:
fit(m, md, 4, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      6.091781   5.975792  
    1      5.748205   5.642408                                                                                         
    2      5.515157   5.394542                                                                                         
    3      5.348775   5.201637                                                                                         



[5.201637]

### GRU

In [140]:
class CharSeqStatefulGRU(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.GRU(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [141]:
# From the pytorch source code - for reference

def GRUCell(input, hidden, w_ih, w_hh, b_ih, b_hh):
    gi = F.linear(input, w_ih, b_ih)
    gh = F.linear(hidden, w_hh, b_hh)
    i_r, i_i, i_n = gi.chunk(3, 1)
    h_r, h_i, h_n = gh.chunk(3, 1)

    resetgate = F.sigmoid(i_r + h_r)
    inputgate = F.sigmoid(i_i + h_i)
    newgate = F.tanh(i_n + resetgate * h_n)
    return newgate + inputgate * (hidden - newgate)

In [142]:
m = CharSeqStatefulGRU(md.nt, n_fac, 512).cuda()

opt = optim.Adam(m.parameters(), 1e-3)

In [143]:
%timeit fit(m, md, 6, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=6), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      6.066504   5.915525  
    1      5.674269   5.56004                                                                                          
    2      5.41742    5.261218                                                                                         
    3      5.18545    5.060897                                                                                         
    4      5.026189   4.863071                                                                                         
    5      4.840787   4.706044                                                                                         



HBox(children=(IntProgress(value=0, description='Epoch', max=6), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      4.71677    4.566166  
    1      4.550214   4.435846                                                                                         
    2      4.390211   4.305636                                                                                         
    3      4.283414   4.205716                                                                                         
    4      4.159799   4.095462                                                                                         
    5      4.076872   3.983641                                                                                         



HBox(children=(IntProgress(value=0, description='Epoch', max=6), HTML(value='')))

...

epoch      trn_loss   val_loss                                                                                         
    0      2.909336   2.954999  
    1      2.853991   2.929542                                                                                         
    2      2.850746   2.903054                                                                                         
    3      2.829871   2.888196                                                                                         
    4      2.77677    2.859377                                                                                         
    5      2.774796   2.822279                                                                                         



HBox(children=(IntProgress(value=0, description='Epoch', max=6), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      2.714619   2.799456  
    1      2.741952   2.800181                                                                                         
    2      2.717779   2.775222                                                                                         
    3      2.670951   2.761652                                                                                         
    4      2.642206   2.73935                                                                                          
    5      2.654712   2.724295                                                                                         



HBox(children=(IntProgress(value=0, description='Epoch', max=6), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      2.595458   2.732153  
    1      2.577773   2.718429                                                                                         
    2      2.581104   2.685483                                                                                         
    3      2.544935   2.662391                                                                                         
    4      2.579764   2.654138                                                                                         
    5      2.566341   2.625561                                                                                         

28.9 s ± 395 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [144]:
set_lrs(opt, 1e-4)

In [145]:
%timeit fit(m, md, 3, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      2.328863   2.47705   
    1      2.299953   2.436171                                                                                         
    2      2.298146   2.39809                                                                                          



HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      2.292751   2.394148  
    1      2.2757     2.375929                                                                                         
    2      2.261224   2.352224                                                                                         



HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

...

epoch      trn_loss   val_loss                                                                                         
    0      2.161767   2.256735  
    1      2.171722   2.244369                                                                                         
    2      2.163695   2.241668                                                                                         



HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      2.138064   2.236043  
    1      2.145211   2.230179                                                                                         
    2      2.129272   2.224459                                                                                         



HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      2.125478   2.219153  
    1      2.125529   2.210223                                                                                         
    2      2.106884   2.213417                                                                                         

14.6 s ± 361 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Putting it all together: LSTM

In [146]:
from fastai import sgdr

n_hidden=512

In [147]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [148]:
m = CharSeqStatefulLSTM(md.nt, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [158]:
PATH='C://Users/sarad/Documents/fastai/courses/dl1/data/SuShi/'
os.makedirs(f'{PATH}models', exist_ok=True)

In [150]:
%timeit fit(m, md, 2, lo.opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      6.059476   6.054818  
    1      5.793957   5.697289                                                                                         



HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      5.648384   5.554955  
    1      5.524454   5.419644                                                                                         



HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

...

epoch      trn_loss   val_loss                                                                                         
    0      5.192591   5.005971  
    1      5.167614   5.02337                                                                                          



HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      5.135167   4.965368  
    1      5.089534   4.95017                                                                                          



HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      5.081523   4.940741  
    1      5.095331   4.886881                                                                                         

16.5 s ± 205 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [151]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      4.902965   4.739222  
    1      5.050755   4.790213                                                                                         
    2      4.832848   4.673209                                                                                         
    3      5.086874   4.830372                                                                                         
    4      4.950373   4.717639                                                                                         
    5      4.786842   4.608561                                                                                         
    6      4.680819   4.54759                                                                                          
    7      5.008823   4.750691                                                                                         
    8  

[4.3769345]

In [152]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_a{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
%timeit fit(m, md, 2**6-1, lo.opt, F.nll_loss, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=63), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      4.451617   4.373536  
    1      4.470408   4.368623                                                                                         
    2      4.448079   4.364522                                                                                         
    3      4.440605   4.359096                                                                                         
    4      4.444331   4.357347                                                                                         
    5      4.452813   4.346957                                                                                         
    6      4.429931   4.349927                                                                                         
    7      4.441357   4.34802                                                                                          
    8  

HBox(children=(IntProgress(value=0, description='Epoch', max=63), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      3.725336   3.593517  
    1      3.615303   3.680045                                                                                         
    2      3.580699   3.692941                                                                                         
    3      3.557394   3.648238                                                                                         
    4      3.52495    3.649722                                                                                         
    5      3.501479   3.612817                                                                                         
    6      3.47859    3.621847                                                                                         
    7      3.446712   3.583463                                                                                         
    8  

HBox(children=(IntProgress(value=0, description='Epoch', max=63), HTML(value='')))

...

epoch      trn_loss   val_loss                                                                                         
    0      1.832719   1.367918  
    1      1.855266   1.36204                                                                                          
    2      1.845428   1.358699                                                                                         
    3      1.57956    1.646949                                                                                         
    4      1.596491   1.666751                                                                                         
    5      1.601598   1.746752                                                                                         
    6      1.591682   1.685524                                                                                         
    7      1.600861   1.69831                                                                                          
    8  

HBox(children=(IntProgress(value=0, description='Epoch', max=63), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      1.624954   1.08813   
    1      1.632728   1.080199                                                                                         
    2      1.634939   1.077639                                                                                         
    3      1.369822   1.407853                                                                                         
    4      1.392343   1.390758                                                                                         
    5      1.374324   1.37154                                                                                          
    6      1.387836   1.438467                                                                                         
    7      1.389984   1.38947                                                                                          
    8  

HBox(children=(IntProgress(value=0, description='Epoch', max=63), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      1.404758   0.861967  
    1      1.39695    0.856255                                                                                         
    2      1.405092   0.848339                                                                                         
    3      1.184224   1.141888                                                                                         
    4      1.206934   1.157745                                                                                         
    5      1.180049   1.205245                                                                                         
    6      1.211885   1.17353                                                                                          
    7      1.188678   1.212981                                                                                         
    8  

around 10s/epoch on GTX1070 (eqv AWS P2)



### Test

In [153]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [154]:
get_next('從前')

'呻'

In [155]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [156]:
print(get_next_n('從前', 50))

從前。日長醉送，前堂雨色，故人歌者】倦年見，篙本走。得之意不也，俗學所有，為補花。）【浮山開閣在禹門，潞神甲，<unk>眼也。然信向過，一絕徒往，故人作《一題歌句》。紅帶小詩樂燭腸不如甜酒。人間膏甚物不為齊安酒在別臥燈瑞月二首】倦看沙荻燈。綠浪翻翻燒，此生晴于竹間。二十六裏桔，玉宮蓋以商楊之蜀，果讀者，乃以詩為晴食矣。詩著宗公黃疲。）何年謝伯健。元茶顛還瀝，使君去歲，而不衰而艱；乎<unk>故不可至次元老韻則許矣初時聞勇而送啼老且魯君二首】去年東吏記西行，未能<unk>分禪師治之謂之器，食一十餘也，問其昌之弼耳。我有持此壽，何曾以遺泣。世間本無失，我來誰能<unk>芝。）老客念前生吞耳，但尋樊子自如兒。只恐且慚殘夢九，詩人不得愁<unk>井放紅衣酒倒庭起倒。主人在心腹有知，此心未繼亦逶忙。良遭未賜心與淺，兩時相迫不轉篇。【古日求南戲再和前篇】舊官古桃花，平生延不還。倦歸入吳繼，清<unk>金坎，入瑞如煙毿。浩疑我何所見，淒涼海轉吞雲山


In [157]:
print(get_next_n('我', 50))

我。雲峰吒暗出煙空，坐覺孤煙影落空。古地中有泉塔中，蓬樓曉夜甘。二樂行人至歌者，城南修竹正無堅。幽心本消不用箱，麥瑕花當同過客。臥聞飛地春何落，收韻收拾魚祀息。煩算更吟終月識，更恨嘉十九筆成。黠陽聊複起薏禦，微言何足莊生殿。【今月十七日五首】夢看隨好來迎背，溪山何處雨春泉。不知鱸浦馬應啼，送君<unk>。誦桃花有詩拾種德不願賢且成而奉奉海都而老得之遺不逐不知此達幾也】翠垂本余無風也。夜酌而回。）聚散瀟坐借。）【郭奉元伯父老顛雖陶司馬何亡矣策兄宗縣太守王宴見八十餘餘複留之韻仍惠耳竹】今不醒居處。仲退原生表，鬱杭之數。）督恨畢考，未見雖不誇。朝退有黃門。我道方聞嵩有小詩，下隱之。）猶含鐵水。）清詩淡將飯倒示至長江今流勝官不遇故複次前韻龍湖之亦至郭達廬，中有李卿卿，故感其詩】浮河又似湯，即客，眠衣未香。）枕飲仍讀書之詩不跋慎因和草書並挺之於二子長孤溪方好久】雲夢浮。蘚書真一寄，貴神仙三篆，極妃，應造


### Test

In [166]:
m.load_cycle('C://Users/sarad/Documents/fastai/courses/dl1/data/SuShi/models/cyc_0.h5)

SyntaxError: EOL while scanning string literal (<ipython-input-166-888ee57a3b62>, line 1)

In [ ]:
print(get_next_n('從前', 400))